In [1]:
import numpy as np
import pandas as pd

from tensorflow.python.keras.wrappers.scikit_learn import KerasClassifier
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, FunctionTransformer, OneHotEncoder, label_binarize
from sklearn.pipeline import Pipeline

2021-07-15 20:46:31.258510: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
abs_path = '/home/ssc/Entwicklung/projects/arguseyes/arguseyes/example_pipelines/datasets/sneakers/'

def decode_image(img_str):
    return np.array([int(val) for val in img_str.split(':')])

train_data = pd.read_csv(f'{abs_path}product_images.csv', converters={'image': decode_image })

In [3]:
product_categories = pd.read_csv(f'{abs_path}product_categories.csv')

In [4]:
with_categories = train_data.merge(product_categories, on='category_id')

categories_to_distinguish = ['Sneaker', 'Ankle boot']

images_of_interest = with_categories[with_categories['category_name'].isin(categories_to_distinguish)]

In [5]:
def normalise_image(images):
    return images / 255.0

def reshape_images(images):
    return np.concatenate(images['image'].values) \
             .reshape(images.shape[0], 28, 28, 1)

def create_cnn():
    model = Sequential([
        Conv2D(filters=64, kernel_size=2, padding='same', activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=2),
        Dropout(0.3),
        Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'),
        MaxPooling2D(pool_size=2),
        Dropout(0.3),
        Flatten(),
        Dense(256, activation='relu'),
        Dropout(0.5),
        Dense(2, activation='softmax')
    ])

    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    return model

pipeline = Pipeline(steps=[
    ('normalisation', FunctionTransformer(normalise_image)),
    ('reshaping', FunctionTransformer(reshape_images)),
    ('model', KerasClassifier(create_cnn))
])

In [6]:
train, test = train_test_split(images_of_interest, test_size=0.2)

y_train = label_binarize(train['category_name'], classes=categories_to_distinguish)
y_test = label_binarize(test['category_name'], classes=categories_to_distinguish)

In [7]:
model = pipeline.fit(train[['image']], y_train)

2021-07-15 20:46:34.405054: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-07-15 20:46:34.431066: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2021-07-15 20:46:34.431309: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: GeForce MX150 computeCapability: 6.1
coreClock: 1.5315GHz coreCount: 3 deviceMemorySize: 1.96GiB deviceMemoryBandwidth: 44.76GiB/s
2021-07-15 20:46:34.431328: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2021-07-15 20:46:34.433716: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcublas.so.11
2021-07-15 20:46:34.433767: I tensorflow/stream_executor/platform/defau

30/30 [==============================] - 1s 25ms/step - loss: 0.2917 - accuracy: 0.8884


In [8]:
model.score(test[['image']], y_test)

8/8 [==============================] - 0s 6ms/step - loss: 0.2556 - accuracy: 0.8950


0.894957959651947